# 📏 Módulo 3.2: Métricas en Aprendizaje No Supervisado
### Curso: **Machine Learning con Python** (IFCD093PO)
**Duración estimada:** 4 horas

---

## 🎯 Objetivos del Módulo

En el módulo anterior, descubrimos el mundo del aprendizaje no supervisado. Pero una pregunta quedó en el aire: **¿cómo sabemos si nuestros resultados son buenos?** Este módulo se dedica a responder esa pregunta, proporcionándote las herramientas para evaluar la calidad de tus modelos de clustering.

Al finalizar, serás capaz de:

- ✅ Comprender la diferencia entre evaluación **extrínseca** (con etiquetas) e **intrínseca** (sin etiquetas).
- ✅ Aplicar e interpretar el **Método del Codo (Elbow Method)** para estimar el número óptimo de clústeres.
- ✅ Calcular e interpretar el **Coeficiente de Silueta**, una de las métricas más importantes para evaluar la cohesión y separación de los clústeres.
- ✅ Utilizar otras métricas intrínsecas como el **Índice de Davies-Bouldin** y el **Índice de Calinski-Harabasz**.
- ✅ Tomar decisiones informadas sobre el número de clústeres (`k`) basándote en evidencia cuantitativa.

**¡Prepárate para añadir rigor científico a tu exploración de datos y validar los patrones que descubras!**

---

## 📚 Tabla de Contenidos

1. [El Desafío de la Evaluación](#1-desafio)
2. [Preparación de Datos para Evaluación](#2-preparacion)
3. [Método del Codo (Elbow Method)](#3-codo)
   - [Inercia: El Concepto Clave](#3.1-inercia)
   - [Implementación y Visualización](#3.2-implementacion-codo)
4. [Coeficiente de Silueta](#4-silueta)
   - [Intuición: Cohesión vs. Separación](#4.1-intuicion-silueta)
   - [Cálculo e Interpretación](#4.2-calculo-silueta)
   - [Visualización de Siluetas](#4.3-visualizacion-silueta)
5. [Otras Métricas Intrínsecas](#5-otras-metricas)
   - [Índice de Davies-Bouldin](#5.1-davies-bouldin)
   - [Índice de Calinski-Harabasz](#5.2-calinski-harabasz)
6. [Comparando Métricas: ¿Cuál Usar?](#6-comparando)
7. [Resumen y Próximos Pasos](#7-resumen)

---

## 🧐 1. El Desafío de la Evaluación <a id='1-desafio'></a>

Como no tenemos etiquetas `y`, no podemos usar métricas como *accuracy*, *precision* o *recall*. La evaluación en aprendizaje no supervisado se divide en dos categorías:

- **Evaluación Extrínseca**: Se usa cuando, por casualidad, tenemos las etiquetas verdaderas (ground truth), pero no las usamos para el entrenamiento. Es útil para comparar algoritmos en un entorno académico, pero raro en la práctica. Ejemplos: *Rand Index*, *Homogeneity Score*.

- **Evaluación Intrínseca**: Es el caso más común. Se evalúa la calidad del clustering basándose únicamente en la estructura de los datos y los clústeres formados. Se centra en dos conceptos:
  - **Cohesión**: ¿Qué tan compactos son los clústeres? (Los puntos dentro de un clúster deben estar cerca entre sí).
  - **Separación**: ¿Qué tan separados están los clústeres entre sí? (Los clústeres diferentes deben estar lejos unos de otros).

**En este notebook, nos centraremos en las métricas de evaluación intrínseca.**

## 🛠️ 2. Preparación de Datos para Evaluación <a id='2-preparacion'></a>

Usaremos un conjunto de datos sintético para poder visualizar y entender fácilmente cómo funcionan las métricas. También cargaremos el famoso dataset `iris` para un caso de uso más realista.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs, load_iris # Cargar conjuntos de datos
from sklearn.cluster import KMeans # Algoritmo de clustering K-Means
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler

# --- Datos Sintéticos ---
# Generamos datos con 4 centros bien definidos para que sea fácil de evaluar
# Parametros:
# n_samples: número de muestras
# n_features: número de características
# centers: número de clusters
# cluster_std: desviación estándar de los clusters
X_blobs, y_blobs = make_blobs(n_samples=500, n_features=2, centers=4, cluster_std=0.8, random_state=42)

# Es buena práctica escalar los datos antes del clustering
scaler_blobs = StandardScaler()
X_blobs_scaled = scaler_blobs.fit_transform(X_blobs)

# --- Datos Iris ---
# Cargamos el conjunto de datos Iris
# Este conjunto tiene 3 clases (tipos de flores) y 4 características
iris = load_iris()
X_iris = iris.data
y_iris = iris.target # Usaremos 'y' solo para comparar al final, no para entrenar

scaler_iris = StandardScaler()
X_iris_scaled = scaler_iris.fit_transform(X_iris)

print("Datos sintéticos (escalados) shape:", X_blobs_scaled.shape)
print("Datos Iris (escalados) shape:", X_iris_scaled.shape)

# Visualicemos los datos sintéticos
plt.figure(figsize=(8, 6))
plt.scatter(X_blobs_scaled[:, 0], X_blobs_scaled[:, 1], s=50)
plt.title("Datos Sintéticos para Evaluación de Clustering")
plt.xlabel("Característica 1 (escalada)")
plt.ylabel("Característica 2 (escalada)")
plt.show()
# Visualicemos los datos Iris (solo las dos primeras características para simplicidad)
plt.figure(figsize=(8, 6))
# Usamos las dos primeras características para visualización
# Esto es solo para simplicidad, ya que Iris tiene 4 características
# En un caso real, podríamos usar técnicas de reducción de dimensionalidad como PCA o t-SNE
# Parametros:
# - X: Datos de entrada
# - y: Etiquetas verdaderas
# - cmap: Mapa de colores
# - s: Tamaño de los puntos
# - c: Color basado en las etiquetas verdaderas
plt.scatter(X_iris_scaled[:, 0], X_iris_scaled[:, 1], c=y_iris, cmap='viridis', s=50)
plt.title("Datos Iris (2 primeras características)")
plt.xlabel("Característica 1 (escalada)")
plt.ylabel("Característica 2 (escalada)")
plt.show()  

# Explicación de los gráficos realizados

### 📊 Explicación de los Gráficos Realizados

1. **Gráfico de Dispersión de los Datos Sintéticos**
    - Este gráfico muestra los datos generados artificialmente con 4 clústeres bien definidos. Cada punto representa una muestra y se visualizan dos características escaladas. Sirve para ver visualmente la estructura de los clústeres antes de aplicar cualquier algoritmo.

2. **Gráfico de Dispersión del Dataset Iris**
    - Aquí se muestran las dos primeras características del famoso dataset Iris, coloreando los puntos según su clase real (`y_iris`). Permite observar cómo se distribuyen las especies de flores en el espacio de características.

Estos gráficos ayudan a entender visualmente la estructura de los datos y a tomar decisiones informadas sobre el número óptimo de clústeres.

---

## 💪 3. Método del Codo (Elbow Method) <a id='3-codo'></a>

Es una de las técnicas más populares y sencillas para estimar el número de clústeres. Se basa en el concepto de **inercia**.

### 3.1 Inercia: El Concepto Clave <a id='3.1-inercia'></a>

La inercia mide qué tan compactos o "apretados" están los grupos (clusters) que ha creado un algoritmo de agrupamiento.

**¿Cómo funciona?**

Imagina que tienes puntos agrupados y cada grupo tiene un centro (centroide). La inercia funciona así:

Para cada punto: calcula su distancia hasta el centro de su grupo.

Eleva esa distancia al cuadrado (para que todas sean positivas y penalizar más las distancias grandes).

Suma todas esas distancias al cuadrado.

**Ejemplo**

- Imagina 3 puntos en un grupo cuyo centro está en (0,0):

    - Punto A está a distancia 2 del centro → contribuye 2² = 4

    - Punto B está a distancia 1 del centro → contribuye 1² = 1

    - Punto C está a distancia 3 del centro → contribuye 3² = 9

    Inercia de este grupo = 4 + 1 + 9 = 14

    Si tienes varios grupos, sumas la inercia de todos ellos.

**¿Para qué sirve?**

- Inercia baja = puntos muy cercanos a sus centros = grupos bien definidos ✓

- Inercia alta = puntos dispersos lejos de sus centros = grupos poco compactos ✗

La idea del método del codo es simple: calculamos la inercia para diferentes números de clústeres (`k`). A medida que `k` aumenta, la inercia siempre disminuirá (en el caso extremo, si cada punto es un clúster, la inercia es 0). Buscamos el punto donde la disminución de la inercia se ralentiza drásticamente, formando un "codo" en el gráfico. Este punto es un buen candidato para el número óptimo de clústeres.

**En definitiva:**

La inercia mide lo “juntos” que están los puntos dentro de cada grupo (clúster). Es como sumar todas las distancias de los puntos a su centro de grupo.

Si la inercia es baja, los puntos están bien agrupados.
Si la inercia es alta, los puntos están muy dispersos.

En el método del codo, probamos diferentes números de grupos y vemos cómo baja la inercia. 

El “codo” del gráfico nos dice cuántos grupos son los ideales: es el punto donde dejar de añadir más grupos ya no mejora mucho la agrupación.

In [ ]:
def plot_elbow_method(X, max_k=10):
    """
    Calcula y grafica la inercia para un rango de k y muestra el método del codo.
    """
    inertias = []
    k_range = range(1, max_k + 1)
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
        
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, inertias, 'bo-')
    plt.xlabel('Número de Clústeres (k)')
    plt.ylabel('Inercia (WCSS)')
    plt.title('Método del Codo para Encontrar k Óptimo')
    plt.xticks(k_range)
    plt.grid(True)
    plt.show()

print("--- Método del Codo para Datos Sintéticos ---")
plot_elbow_method(X_blobs_scaled)

print("\n--- Método del Codo para Datos Iris ---")
plot_elbow_method(X_iris_scaled)

**Análisis de los Gráficos:**

- **Datos Sintéticos**: Se observa un "codo" muy claro en `k=4`. Después de este punto, la disminución de la inercia es mucho menos pronunciada. Esto sugiere que 4 es el número óptimo de clústeres, lo cual coincide con cómo generamos los datos.
- **Datos Iris**: El codo es menos pronunciado, pero parece estar en `k=3`. Esto también coincide con el número real de especies de flores en el dataset.

**Limitación**: El método del codo puede ser ambiguo si el "codo" no es claro. Por eso, es bueno complementarlo con otras métricas.

---

## 📐 4. Coeficiente de Silueta <a id='4-silueta'></a>

El Coeficiente de Silueta es una métrica más robusta que mide qué tan bien se ajusta cada punto a su propio clúster en comparación con los clústeres vecinos.

### 4.1 Intuición: Cohesión vs. Separación <a id='4.1-intuicion-silueta'></a>

Para cada punto de datos, el coeficiente de silueta se calcula así:

1.  **`a` (Cohesión)**: La distancia promedio del punto a todos los demás puntos en el **mismo clúster**.
2.  **`b` (Separación)**: La distancia promedio del punto a todos los puntos en el **clúster más cercano que no es el suyo**.

El coeficiente de silueta `s` para un solo punto es:

$$ s = \frac{b - a}{\max(a, b)} $$

El valor de `s` varía entre -1 y 1:
- **`s` cercano a +1**: ¡Excelente! El punto está muy bien agrupado. Está lejos de los clústeres vecinos (`b` es grande) y cerca de los miembros de su propio clúster (`a` es pequeño).
- **`s` cercano a 0**: El punto está en el borde entre dos clústeres. No está claro a cuál pertenece.
- **`s` cercano a -1**: ¡Muy mal! El punto probablemente ha sido asignado al clúster equivocado. Está más cerca de un clúster vecino que del suyo.

El **Silhouette Score** para un conjunto de clústeres es simplemente el promedio de los coeficientes de silueta de todos los puntos.

**Es decir:**

El coeficiente de silueta te dice lo bien que está cada punto dentro de su grupo (clúster):

Si el valor es cercano a 1, el punto está muy bien en su grupo y lejos de los otros grupos (¡perfecto!).

Si el valor es cercano a 0, el punto está entre dos grupos (no está claro a cuál pertenece).

Si el valor es negativo, el punto está más cerca de otro grupo que del suyo (probablemente está mal clasificado).

**En resumen:** cuanto más alto el coeficiente de silueta, mejor agrupado está el punto.

In [ ]:
def plot_silhouette_score(X, max_k=10):
    """
    Calcula y grafica el Silhouette Score para un rango de k.
    """
    silhouette_scores = []
    k_range = range(2, max_k + 1) # Silhouette no está definido para k=1, por eso empezamos en 2
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        kmeans.fit(X)
        score = silhouette_score(X, kmeans.labels_)
        silhouette_scores.append(score)
        
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, silhouette_scores, 'go-')
    plt.xlabel('Número de Clústeres (k)')
    plt.ylabel('Silhouette Score Promedio')
    plt.title('Análisis de Silueta para Encontrar k Óptimo')
    plt.xticks(k_range)
    plt.grid(True)
    plt.show()

print("--- Análisis de Silueta para Datos Sintéticos ---")
plot_silhouette_score(X_blobs_scaled)

print("\n--- Análisis de Silueta para Datos Iris ---")
plot_silhouette_score(X_iris_scaled)

**Análisis de los Gráficos:**

- **Datos Sintéticos**: El Silhouette Score más alto se alcanza en `k=4`. Esto confirma fuertemente el resultado del método del codo.
- **Datos Iris**: El score más alto se da en `k=2`. Esto es interesante y diferente del método del codo. Muestra que, según esta métrica, la mejor separación se logra al dividir los datos en dos grandes grupos. El score para `k=3` también es alto, lo que lo convierte en un candidato viable. Esto resalta la importancia de no depender de una sola métrica.

**¿Por qué la diferencia en Iris?** El dataset Iris tiene dos especies que son muy similares entre sí y una que es muy distinta. El Silhouette Score para `k=2` probablemente agrupa las dos especies similares y separa la distinta, logrando una separación y cohesión global muy buena.

### 4.3 Visualización de Siluetas <a id='4.3-visualizacion-silueta'></a>

Podemos ir un paso más allá y visualizar los coeficientes de silueta para cada punto dentro de cada clúster. Esto nos da una idea de la "salud" de cada clúster.

In [ ]:
from sklearn.metrics import silhouette_samples
import matplotlib.cm as cm

def plot_silhouette_diagram(X, k):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    cluster_labels = kmeans.fit_predict(X)
    
    silhouette_avg = silhouette_score(X, cluster_labels)
    sample_silhouette_values = silhouette_samples(X, cluster_labels)
    
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(10, 7)
    
    ax1.set_xlim([-0.2, 1])
    ax1.set_ylim([0, len(X) + (k + 1) * 10])
    
    y_lower = 10
    for i in range(k):
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        color = cm.nipy_spectral(float(i) / k)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)
        
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10
        
    ax1.set_title(f"Diagrama de Silueta para k={k}")
    ax1.set_xlabel("Coeficiente de Silueta")
    ax1.set_ylabel("Clúster")
    
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax1.set_yticks([])
    plt.show()

print("--- Diagrama de Silueta para Datos Sintéticos con k=4 ---")
plot_silhouette_diagram(X_blobs_scaled, 4)

print("\n--- Diagrama de Silueta para Datos Iris con k=2 ---")
plot_silhouette_diagram(X_iris_scaled, 2)

print("\n--- Diagrama de Silueta para Datos Iris con k=3 ---")
plot_silhouette_diagram(X_iris_scaled, 3)

**Análisis de los Diagramas:**

- **Datos Sintéticos (k=4)**: Todos los clústeres tienen un grosor similar y la mayoría de sus puntos están por encima del promedio (línea roja). Esto indica un clustering muy bueno y equilibrado.
- **Iris (k=2)**: Un clúster es grande y el otro más pequeño, pero ambos están claramente por encima del promedio. Es una buena partición.
- **Iris (k=3)**: Se ve que el clúster 0 es muy bueno (ancho y muy por encima del promedio). Sin embargo, los clústeres 1 y 2 son más estrechos y están más cerca de la línea promedio. Esto visualiza por qué el score para k=3 es bueno, pero no tan alto como para k=2.

---

## 📈 5. Otras Métricas Intrínsecas <a id='5-otras-metricas'></a>

Existen otras métricas que también evalúan la cohesión y separación, pero con formulaciones matemáticas diferentes.

### 5.1 Índice de Davies-Bouldin <a id='5.1-davies-bouldin'></a>

**¿Qué mide?**

Evalúa qué tan bien separados están tus grupos comparando cada grupo con su "vecino más parecido".

**¿Cómo funciona?**

Para cada grupo, el índice hace dos preguntas:

-   ¿Qué tan compacto es mi grupo? → Mide qué tan cerca están los puntos de su centro
-   ¿Qué tan lejos está del grupo más cercano? → Mide la distancia entre centros

**Luego calcula:**

    Similitud = (Compacidad grupo A + Compacidad grupo B) / Distancia entre centros

**Ejemplo intuitivo**

Imagina dos grupos de personas:

1. *Escenario 1 (MALO - Davies-Bouldin alto):*

- Grupo A: personas muy dispersas (compacidad alta)

- Grupo B: muy cerca del Grupo A (distancia baja)

Resultado: grupos confusos, mal separados

2. *Escenario 2 (BUENO - Davies-Bouldin bajo):*

- Grupo A: personas muy juntas (compacidad baja)

- Grupo B: muy lejos del Grupo A (distancia alta)

Resultado: grupos claros y bien definidos

**Interpretación**

- Valor cercano a 0 = Grupos perfectamente separados ✓

- Valor alto = Grupos confusos o solapados ✗

**Ventaja práctica**

Es más rápido de calcular que otros índices como el Silhouette Score, por lo que es útil cuando trabajas con muchos datos.

### 5.2 Índice de Calinski-Harabasz <a id='5.2-calinski-harabasz'></a>

**¿Qué mide?**

Compara qué tan separados están los grupos entre sí versus qué tan compactos son internamente.

**¿Cómo funciona?**

Es básicamente una división:

    Calinski-Harabasz = Separación entre grupos / Compacidad dentro de grupos

*Numerador (arriba): ¿Qué tan alejados están los centros de los grupos?*

*Denominador (abajo): ¿Qué tan dispersos están los puntos dentro de cada grupo?*

**Ejemplo** 

Imagina clasificar animales en grupos:

1. Escenario 1 (BUENO - valor alto):

-   Los gatos están muy juntos entre sí (compacidad baja ✓)

-   Los perros están muy juntos entre sí (compacidad baja ✓)

-   Gatos y perros están MUY separados (separación alta ✓)

*Resultado: Valor alto = excelente agrupamiento*

2. Escenario 2 (MALO - valor bajo):

-   Los gatos están dispersos (compacidad alta ✗)

-   Los perros están dispersos (compacidad alta ✗)

-   Gatos y perros están mezclados (separación baja ✗)

*Resultado: Valor bajo = mal agrupamiento*

**Interpretación**

- Valor alto = Grupos densos y bien separados ✓

- Valor bajo = Grupos dispersos o solapados ✗

In [ ]:
def compare_all_metrics(X, max_k=10):
    k_range = range(2, max_k + 1)
    results = []

    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        labels = kmeans.fit_predict(X)
        
        silhouette = silhouette_score(X, labels)
        davies_bouldin = davies_bouldin_score(X, labels)
        calinski_harabasz = calinski_harabasz_score(X, labels)
        
        results.append({
            'k': k,
            'Silhouette (Más alto es mejor)': silhouette,
            'Davies-Bouldin (Más bajo es mejor)': davies_bouldin,
            'Calinski-Harabasz (Más alto es mejor)': calinski_harabasz
        })

    return pd.DataFrame(results).set_index('k')

print("--- Comparación de Métricas para Datos Sintéticos ---")
results_blobs = compare_all_metrics(X_blobs_scaled)
display(results_blobs)

print("\n--- Comparación de Métricas para Datos Iris ---")
results_iris = compare_all_metrics(X_iris_scaled)
display(results_iris)

---

## 🤔 6. Comparando Métricas: ¿Cuál Usar? <a id='6-comparando'></a>

**Análisis de las Tablas:**

- **Datos Sintéticos**: Todas las métricas coinciden. El Silhouette y Calinski-Harabasz tienen su máximo en `k=4`, y el Davies-Bouldin tiene su mínimo en `k=4`. ¡Consenso total!

- **Datos Iris**: Todas las métricas coinciden.
  - **Silhouette** prefiere `k=2`.
  - **Davies-Bouldin** prefiere `k=2` (su valor más bajo es 0.593).
  - **Calinski-Harabasz** prefiere `k=2` (su valor más alto es 251.34).

**Conclusión Práctica:**

1.  **Empieza con el Método del Codo**: Es una primera aproximación rápida y visual.
2.  **Confirma con Silhouette Score**: Es la métrica más intuitiva y robusta. Su visualización por clúster es muy informativa.
3.  **Usa Davies-Bouldin y Calinski-Harabasz como desempate o confirmación**: Son rápidas y pueden ofrecer una perspectiva diferente.
4.  **El contexto de negocio es el rey**: Si las métricas no se ponen de acuerdo, la decisión final debe basarse en qué número de clústeres tiene más sentido para tu problema. ¿Es más útil para marketing tener 2 segmentos de clientes o 3? La respuesta a esa pregunta puede ser más importante que una décima de diferencia en una métrica.

---

## 📝 7. Resumen y Próximos Pasos <a id='7-resumen'></a>

### 🎉 ¡Ahora puedes medir la calidad de tus descubrimientos!

#### ✅ Lo que has aprendido:

1. **El Principio Fundamental**
   - La evaluación intrínseca se basa en la **cohesión** (qué tan juntos están los puntos de un clúster) y la **separación** (qué tan lejos están los clústeres entre sí).

2. **Tus Herramientas de Evaluación**
   - **Método del Codo**: Una forma visual y rápida de estimar `k` basada en la inercia.
   - **Coeficiente de Silueta**: La métrica estrella. Un valor alto (cercano a 1) es bueno. Mide cohesión y separación para cada punto.
   - **Índice de Davies-Bouldin**: Un valor bajo (cercano a 0) es bueno.
   - **Índice de Calinski-Harabasz**: Un valor alto es bueno.

3. **La Estrategia Correcta**
   - No te fíes de una sola métrica. Úsalas en conjunto para tomar una decisión informada.
   - Recuerda que estas métricas son una guía. La validación final a menudo requiere conocimiento del dominio y análisis cualitativo.

---

### 🚀 Próximo Módulo: Algoritmos de Clustering

Ya sabes qué es el clustering y cómo evaluar sus resultados. Es hora de sumergirse en los algoritmos que hacen la magia.

En el próximo módulo, exploraremos en detalle:

- **K-Means**: El algoritmo de clustering más famoso y utilizado.
- **Clustering Jerárquico**: Un enfoque que crea una jerarquía de clústeres.
- **DBSCAN**: Un algoritmo basado en densidad, capaz de encontrar clústeres de formas arbitrarias y manejar el ruido.

**Has aprendido a definir el objetivo (clustering) y a medir el éxito (métricas). Ahora, ¡vamos a aprender a construir los motores que lo hacen posible!**